In [2]:
import pandas as pd 

In [3]:
import os

In [7]:
print(os.listdir("../my-flask-app"))

['.ipynb_checkpoints', 'aldo.ipynb', 'algo.py', 'app.py', 'ForumServer.py', 'friends.py', 'loginandsignup.py', 'logintest.py', 'package.json', 'postcreation.py', 'poststesting.py', 'tempPython.py', 'uploaddatabase.py', 'uploads', 'uploadstory.py', 'venv', '__pycache__']


In [8]:
talent = pd.read_csv("talent.csv")

In [9]:
talent.columns

Index(['id', 'name', 'price', 'reactions', 'stars', 'joined', 'categories',
       'in_13_reasons_why', 'in_90_day_fiance', 'in_actors',
       ...
       'in_ufc', 'in_vanderpump_rules', 'in_venture_capitalists', 'in_viners',
       'in_vlog_squad', 'in_voice_actors', 'in_winter_sports', 'in_writers',
       'in_younow', 'in_youtubers'],
      dtype='object', length=110)